In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3 as lite
from sqlite3 import Error
from pathlib import Path
from datetime import date
import numpy as np
import matplotlib.ticker as tick
import requests
import difflib as diff
import re 
import csv
import ast
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
%matplotlib inline

# pd.set_option('mode.chained_assignment', None)

def create_connection(db_file):
    """
    create a connection to sqlite3 database
    """
    conn = None
    try:
        conn = lite.connect(db_file, timeout=10)  # connection via sqlite3
    except Error as e:
        print(e)
    return conn

conn = create_connection("../CVEfixes.db")

query = """
    SELECT m.code, m.before_change, c.committer_date
    FROM file_change f, method_change m, commits c
    WHERE m.file_change_id = f.file_change_id
    AND c.hash = f.hash
    AND f.programming_language = 'PHP';
"""

df = pd.read_sql_query(query, conn)
df

,code,before_change,committer_date
0,\tfunction sf_downloads() {\n\t\t$sf_prefix = ...,True,2005-10-03 05:40:29+00:00
1,\tfunction sf_downloads() {\n\t\t$sf_prefix = ...,False,2005-10-03 05:40:29+00:00
2,\t\t\t\t$this->{$var} = trim($_REQUEST[$var]);...,True,2006-09-17 06:38:32+00:00
3,\t\t\t\t$this->{$var} = wp_specialchars(trim($...,False,2006-09-17 06:38:32+00:00
4,\t\t\t$email = strtolower($comment_author_emai...,True,2006-09-17 06:38:32+00:00
...,...,...,...
28252,function get_tracks($type)\n{\n switch ($ty...,False,2024-07-13 19:26:57+07:00
28253,public function get_sessiondata()\n {\n...,False,2024-07-13 19:26:57+07:00
28254,public function get_sessiondata()\n {\n...,True,2024-07-13 19:26:57+07:00
28255,public function set_session_cookies($user_...,False,2024-07-13 19:26:57+07:00


In [2]:
df = df.drop_duplicates(subset=['code'], ignore_index=True)
df = df.rename(columns={'before_change': 'label', 'code': 'text'})
df.loc[df.label == 'False', 'label'] = 0
df.loc[df.label == 'True', 'label'] = 1
df

,text,label,committer_date
0,\tfunction sf_downloads() {\n\t\t$sf_prefix = ...,1,2005-10-03 05:40:29+00:00
1,\tfunction sf_downloads() {\n\t\t$sf_prefix = ...,0,2005-10-03 05:40:29+00:00
2,\t\t\t\t$this->{$var} = trim($_REQUEST[$var]);...,1,2006-09-17 06:38:32+00:00
3,\t\t\t\t$this->{$var} = wp_specialchars(trim($...,0,2006-09-17 06:38:32+00:00
4,\t\t\t$email = strtolower($comment_author_emai...,1,2006-09-17 06:38:32+00:00
...,...,...,...
20393,function get_tracks($type)\n{\n switch ($ty...,0,2024-07-13 19:26:57+07:00
20394,public function get_sessiondata()\n {\n...,0,2024-07-13 19:26:57+07:00
20395,public function get_sessiondata()\n {\n...,1,2024-07-13 19:26:57+07:00
20396,public function set_session_cookies($user_...,0,2024-07-13 19:26:57+07:00


In [3]:
print('Vulnerable:', len(df[df['label'] == 1]))
print('Not vulnerable:', len(df[df['label'] == 0]))

Vulnerable: 4437
Not vulnerable: 15961


In [7]:
# Save dataset without preprocessing ready for training
from sklearn.model_selection import train_test_split

df_nopreproc = df.drop(df.columns[[2]], axis=1)

train, test = train_test_split(df_nopreproc, test_size=0.2)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
test, validation = train_test_split(test, test_size=0.5)
train.to_json('php_nopreproc_train.json', orient='records')
validation.to_json('php_nopreproc_valid.json', orient='records')
test.to_json('php_nopreproc_test.json', orient='records')

In [13]:
# Balance dataset

# Randomly drop non vulnerable code until it reaches a 50/50 split
to_drop = len(df[df['label'] == 0]) - len(df[df['label'] == 1])
frac_to_drop = to_drop/len(df[df['label'] == 0])
df_balanced = df.drop(df[df['label'] == 0].sample(frac=frac_to_drop).index)
df_balanced = df_balanced.reset_index(drop=True)
df_balanced = df_balanced.drop(df.columns[[2]], axis=1)

# Split and save 80/10/10
train, test = train_test_split(df_balanced, test_size=0.2)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
test, validation = train_test_split(test, test_size=0.5)
train.to_json('php_balanced_train.json', orient='records')
validation.to_json('php_balanced_valid.json', orient='records')
test.to_json('php_balanced_test.json', orient='records')

In [24]:
# Prevent time traveling

#df['committer_date'] = pd.to_datetime(df['committer_date'])
df = df.sort_values(by='committer_date')
dfx = df.drop(df.columns[[2]], axis=1)
split_index = int(len(df) * 0.8)
train = dfx.iloc[:split_index]
test = dfx.iloc[split_index:]
test, validation = train_test_split(test, test_size=0.5)
train.to_json('php_date_train.json', orient='records')
validation.to_json('php_date_valid.json', orient='records')
test.to_json('php_date_test.json', orient='records')

In [22]:
!pwd

/lunarc/nobackup/projects/lu2024-17-13/kevin/php
